In [1]:
import osmnx as ox
import pandas as pd
import numpy as np
import re

In [2]:
# Example: get driving network for a city
G = ox.graph_from_place("Somerville, Massachusetts, USA", network_type="drive")

In [3]:
# project graph to an appropriate UTM CRS
# G_proj = ox.project_graph(G)

In [4]:
# # simplify intersections
# G2 = ox.simplification.consolidate_intersections(
#     G_proj,
#     rebuild_graph=True,
#     tolerance=15,
#     dead_ends=False,
# )
# len(G2)

In [5]:
# Convert to GeoDataFrames (nodes + edges)
nodes, edges = ox.graph_to_gdfs(G)

In [6]:
edges[['highway', 'name', 'maxspeed']].sample(10)

,,,highway,name,maxspeed
u,v,key,,,
71950873,61172077,0,residential,Newman Place,25 mph
71924003,71936966,0,residential,Alpine Street,20 mph
61171240,61173310,0,secondary,Highland Avenue,20 mph
71924890,71936634,0,residential,Otis Street,20 mph
71940999,71933814,0,residential,Wellington Avenue,25 mph
71941036,71917425,0,tertiary,Morrison Avenue,20 mph
71952579,71923476,0,residential,Hillsdale Road,NaN
71953235,71949504,0,residential,Linden Avenue,20 mph
61170761,71941962,0,residential,Hillside Avenue,25 mph


In [7]:
def extract_maxspeed(x):
    # Handle NaN early
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return np.nan

    # Handle lists and numpy arrays uniformly
    if isinstance(x, (list, np.ndarray)):
        speeds = [extract_maxspeed(i) for i in x]
        # filter out NaNs
        speeds = [s for s in speeds if not pd.isna(s)]
        return max(speeds) if speeds else np.nan

    # Convert to string and extract digits
    x_str = str(x)
    nums = re.findall(r'\d+', x_str)

    return int(max(nums, key=int)) if nums else np.nan

In [8]:
edges['maxspeed_int'] = edges['maxspeed'].apply(extract_maxspeed)

In [9]:
edges[['highway', 'name', 'maxspeed', 'maxspeed_int']].sample(10)

,,,highway,name,maxspeed,maxspeed_int
u,v,key,,,,
71951018,71928807,0,residential,Stickney Avenue,20 mph,20.0
61173051,71938041,0,residential,Dickson Street,NaN,NaN
71916774,71953040,0,residential,Victoria Street,NaN,NaN
71954205,945798791,0,primary,Somerville Avenue,25 mph,25.0
61182360,61178893,0,secondary,Medford Street,25 mph,25.0
61177298,11395248465,0,residential,Central Road,25 mph,25.0
604063583,71936370,0,residential,Jaques Street,20 mph,20.0
71920826,71947231,0,secondary,Broadway,25 mph,25.0
71919864,71949322,0,residential,Century Street,20 mph,20.0


In [11]:
edges.to_file("../data/somerville_roads.gpkg", layer="roads", driver="GPKG")

## Bike

In [18]:
# Example: get driving network for a city
G = ox.graph_from_place("Somerville, Massachusetts, USA", network_type="bike")

In [19]:
# Convert to GeoDataFrames (nodes + edges)
nodes, edges = ox.graph_to_gdfs(G)

In [17]:
edges.to_file("../data/somerville_bike.gpkg", layer="roads", driver="GPKG")